In [ ]:
# import FormanRicci, FormanRicci4, and OllivierRicci from GraphRicciCurvature
# import networkx as nx

import numpy as np
import networkx as nx

# import module to take time measurements
import time

from GraphRicciCurvature.FormanRicci import FormanRicci
from GraphRicciCurvature.OllivierRicci import OllivierRicci
from GraphRicciCurvature.FormanRicci4 import FormanRicci4

In [ ]:
# create an SBM graph with 100 nodes and 2 communities, with 0.5 probability of an edge within a community and 0.1 probability of an edge between communities

times_dict = {'borf': {}, 'barf_3': {}, 'barf_4': {}}

for k in range(1, 21):
    times_dict['borf'][100*k] = []
    times_dict['barf_3'][100*k] = []
    times_dict['barf_4'][100*k] = []

    for _ in range(10):
        G = nx.planted_partition_graph(2, k*100, 0.5, 0.1)

        # compute the times for borf, barf_3, and barf_4
        borf_time = borf(G, loops=1, is_undirected=True, batch_add=50, batch_remove=50)[1]
        barf_3_time = barf_3(G, loops=1, is_undirected=True, batch_add=50, batch_remove=50)[1]
        barf_4_time = barf_4(G, loops=1, is_undirected=True, batch_add=50, batch_remove=50)[1]

        # append the times to the dictionary
        times_dict['borf'][100*k].append(borf_time)
        times_dict['barf_3'][100*k].append(barf_3_time)
        times_dict['barf_4'][100*k].append(barf_4_time)

In [ ]:
# plot the means and standard deviations of the times
import matplotlib.pyplot as plt

for key in times_dict:
    means = []
    stds = []
    for k in range(1, 21):
        means.append(np.mean(times_dict[key][100*k]))
        stds.append(np.std(times_dict[key][100*k]))

    plt.errorbar(range(1, 21), means, yerr=stds, label=key)

plt.xlabel('Number of nodes per community')
plt.ylabel('Time (s)')
plt.legend()
plt.show()

In [ ]:
# definitions of borf, barf_3, and barf_4

def borf(graph, loops=10, remove_edges=True, removal_bound=0.5, tau=1,
    is_undirected=False, batch_add=4, batch_remove=2, device=None,
    save_dir='rewired_graphs', dataset_name=None, graph_index=0, debug=False):
    
    # create a copy of the graph
    G = graph.copy()

    # Rewiring begins
    start_time = time.time()

    for _ in range(loops):
    # for _ in range(5):
        # Compute ORC
        orc = OllivierRicci(G, alpha=0)
        orc.compute_ricci_curvature()
        _C = sorted(orc.G.edges, key=lambda x: orc.G[x[0]][x[1]]['ricciCurvature']['rc_curvature'])

        # Get top positive curved edges
        most_pos_edges = _C[-batch_remove:]

        # get all edges with negative curvature
        # most_neg_edges = [edge for edge in _C if orc.G[edge[0]][edge[1]]['ricciCurvature']['rc_curvature'] < 0]
        most_neg_edges = _C[:batch_add]

        # Add edges
        for (u, v) in most_neg_edges:
            pi = orc.G[u][v]['ricciCurvature']['rc_transport_cost']
            p, q = np.unravel_index(pi.values.argmax(), pi.values.shape)
            p, q = pi.index[p], pi.columns[q]
            
            if(p != q and not G.has_edge(p, q)):
                G.add_edge(p, q)

        # Remove edges
        for (u, v) in most_pos_edges:
            if(G.has_edge(u, v)):
                G.remove_edge(u, v)

        end_time = time.time()
        rewiring_duration = end_time - start_time

    return G, rewiring_duration


def barf_3(graph, loops=10, remove_edges=True, is_undirected=False, batch_add=4, batch_remove=2, 
          device=None, save_dir='rewired_graphs', dataset_name=None, graph_index=0, debug=False):

    # create a copy of the graph
    G = graph.copy()

    # Rewiring begins
    start_time = time.time()
    
    for _ in range(loops):
    # for _ in range(5):
        try:
            # Compute AFRC
            afrc = FormanRicci(G)
            afrc.compute_ricci_curvature()
            _C = sorted(afrc.G.edges, key=lambda x: afrc.G[x[0]][x[1]]['AFRC'])

            # convert _C to a numpy array
            curv_vals = np.array(_C)

            # find the threshold
            # if current_iteration == 0:
            #    threshold = _find_threshold(curv_vals)
            #    print('Threshold: %f' % threshold)

            # Get top negative and positive curved edges
            most_pos_edges = _C[-batch_remove:]

            # most_neg_edges = [edge for edge in _C if afrc.G[edge[0]][edge[1]]['AFRC'] < threshold]
            most_neg_edges = _C[:batch_add]

            # Remove edges
            for (u, v) in most_pos_edges:
                if(G.has_edge(u, v)):
                    G.remove_edge(u, v)

            # Add edges
            for (u, v) in most_neg_edges:
                # if there is a neighbor w of u that is not a neighbor of v,
                # choose w at random add an edge between v and w
                if list(set(G.neighbors(u)) - set(G.neighbors(v))) != []:
                    w = np.random.choice(list(set(G.neighbors(u)) - set(G.neighbors(v))))
                    G.add_edge(v, w)
                    # add attributes "AFRC", "triangles", and "weight" to each added edge
                    G[v][w]["AFRC"] = 0.0
                    G[v][w]["triangles"] = 0
                    G[v][w]["weight"] = 1.0

                # else if there is a neighbor w of v that is not a neighbor of u,
                # choose w at random add an edge between u and w
                elif list(set(G.neighbors(v)) - set(G.neighbors(u))) != []:
                    w = np.random.choice(list(set(G.neighbors(v)) - set(G.neighbors(u))))
                    G.add_edge(u, w)
                    # add attributes "AFRC", "triangles", and "weight" to each added edge
                    G[u][w]["AFRC"] = 0.0
                    G[u][w]["triangles"] = 0
                    G[u][w]["weight"] = 1.0

                # else if all neighbors of u are neighbors of v, and vice versa,
                # do nothing
                else:
                    pass

        except ValueError:
            continue

        end_time = time.time()
        rewiring_duration = end_time - start_time

    return G, rewiring_duration


def barf_4(graph, loops=10, remove_edges=True, is_undirected=False, batch_add=4, batch_remove=2, 
          device=None, save_dir='rewired_graphs', dataset_name=None, graph_index=0, debug=False):

    # create a copy of the graph
    G = graph.copy()

    # Rewiring begins
    start_time = time.time()

    for _ in range(loops):
    # for _ in range(5):
        try:
            # Compute AFRC
            afrc = FormanRicci4(G)
            afrc.compute_afrc_4()
            _C = sorted(afrc.G.edges, key=lambda x: afrc.G[x[0]][x[1]]['AFRC_4'])

            # convert _C to a numpy array
            curv_vals = np.array(_C)

            # find the threshold
            # threshold = _find_threshold(curv_vals)
            # print('Threshold: %f' % threshold)

            # Get top negative and positive curved edges
            most_pos_edges = _C[-batch_remove:]
            
            # most_neg_edges = [edge for edge in _C if afrc.G[edge[0]][edge[1]]['AFRC_4'] < threshold]
            most_neg_edges = _C[:batch_add]

            # Remove edges
            for (u, v) in most_pos_edges:
                if(G.has_edge(u, v)):
                    G.remove_edge(u, v)

            # Add edges
            for (u, v) in most_neg_edges:
                # if there is a neighbor w of u that is not a neighbor of v,
                # choose w at random add an edge between v and w
                if list(set(G.neighbors(u)) - set(G.neighbors(v))) != []:
                    w = np.random.choice(list(set(G.neighbors(u)) - set(G.neighbors(v))))
                    G.add_edge(v, w)
                    # add attributes "AFRC", "triangles", and "weight" to each added edge
                    G[v][w]["AFRC"] = 0.0
                    G[v][w]["triangles"] = 0
                    G[v][w]["weight"] = 1.0
                    G[v][w]["AFRC_4"] = 0.0
                    G[v][w]["quadrangles"] = 0

                # else if there is a neighbor w of v that is not a neighbor of u,
                # choose w at random add an edge between u and w
                elif list(set(G.neighbors(v)) - set(G.neighbors(u))) != []:
                    w = np.random.choice(list(set(G.neighbors(v)) - set(G.neighbors(u))))
                    G.add_edge(u, w)
                    # add attributes "AFRC", "triangles", and "weight" to each added edge
                    G[u][w]["AFRC"] = 0.0
                    G[u][w]["triangles"] = 0
                    G[u][w]["weight"] = 1.0
                    G[u][w]["AFRC_4"] = 0.0
                    G[u][w]["quadrangles"] = 0

                # else if all neighbors of u are neighbors of v, and vice versa,
                # do nothing
                else:
                    pass

        except ValueError:
            # if there are no edges with negative curvature, do nothing
            continue

        end_time = time.time()
        rewiring_duration = end_time - start_time

    return G, rewiring_duration